### FRED Data Importation 

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from fredapi import Fred

# --- Define Parameters ---
GREEN_TICKERS = ['ENPH', 'TSLA', 'NEE']
BROWN_TICKERS = ['XOM', 'CVX', 'EOG']
START_DATE = '2015-01-01'
END_DATE = '2025-10-31'
RISK_FREE_SERIES_ID = 'DGS3MO'

# --- Get Stock Data (To define the definitive set of trading days) ---
all_tickers = GREEN_TICKERS + BROWN_TICKERS

# download raw data (structure may vary depending on number of tickers / yfinance version)
raw_price = yf.download(all_tickers, start=START_DATE, end=END_DATE)

# Robustly extract adjusted close prices (handle single-level and multi-level column cases)
if isinstance(raw_price.columns, pd.MultiIndex):
	cols = raw_price.columns
	lv0 = cols.get_level_values(0)
	lv1 = cols.get_level_values(1)
	# mask columns where any level equals 'Adj Close'
	mask = (lv0 == 'Adj Close') | (lv1 == 'Adj Close')
	if mask.any():
		price_data = raw_price.loc[:, mask]
		# flatten to ticker symbols as columns
		if isinstance(price_data.columns, pd.MultiIndex):
			# detect which level contains tickers (compare against all_tickers)
			if any(item in all_tickers for item in lv0):
				ticker_level = 0
			elif any(item in all_tickers for item in lv1):
				ticker_level = 1
			else:
				ticker_level = 1
			price_data.columns = price_data.columns.get_level_values(ticker_level)
	else:
		# if 'Adj Close' not found in any level, assume raw_price already contains prices per ticker
		price_data = raw_price
else:
	# single-level columns: assume columns are tickers with adjusted close prices
	price_data = raw_price

# Ensure chronological order and compute returns
price_data = price_data.sort_index()
returns_data = price_data.pct_change().dropna()
trading_days_index = returns_data.index  # This is the set of dates we MUST use

# Initialize FRED (Replace 'YOUR_FRED_API_KEY')
fred = Fred(api_key='2f3c0255e16774ca2f936a70d6556b00')


/var/folders/00/c54qzf757tzdyqs5hr4n94vc0000gn/T/ipykernel_36073/2122135617.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_price = yf.download(all_tickers, start=START_DATE, end=END_DATE)
[*********************100%***********************]  6 of 6 completed


### FRED Data Cleaning Alignment 

In [2]:
# ---. Download and Transform FRED Data ---

# Download the annual percentage rate (DGS3MO)
rf_data_annual_percent = fred.get_series(
    RISK_FREE_SERIES_ID,
    observation_start=START_DATE,
    observation_end=END_DATE
)

# Convert to Daily Decimal Rate: Annual Percent -> Annual Decimal -> Daily Decimal
# (Dividing by 252 is necessary to calculate the daily excess return for the Sharpe ratio/regression)
rf_daily_decimal = (rf_data_annual_percent / 100) / 252 

# Forward Fill (FFILL): Carries the last observed rate forward, including weekends/holidays.
# This fills in the gaps between the dates FRED publishes.
rf_daily_decimal_filled = rf_daily_decimal.ffill()

# --- Final Alignment (The most important cleaning step) ---

# Reindex the FRED data to match the exact trading days from your stock data.
# This ensures that for every stock return observation, there is a corresponding risk-free rate.
rf_aligned = rf_daily_decimal_filled.reindex(trading_days_index)

# Drop any potential remaining NaNs (Should only be a few at the very start, if any)
rf_aligned = rf_aligned.dropna()

print("### Cleaned and Aligned Daily Risk-Free Rate Data ###")
print(rf_aligned.head())
print(f"\nTotal trading days in return data: {len(trading_days_index)}")
print(f"Total aligned risk-free rate data points: {len(rf_aligned)}")


### Cleaned and Aligned Daily Risk-Free Rate Data ###
Date
2015-01-05    1.190476e-06
2015-01-06    1.190476e-06
2015-01-07    1.190476e-06
2015-01-08    1.190476e-06
2015-01-09    7.936508e-07
dtype: float64

Total trading days in return data: 2723
Total aligned risk-free rate data points: 2723


# Saved rf_daily.csv

In [3]:
rf_aligned = rf_aligned.copy()
# save the aligned risk-free series (use the existing variable name)
rf_aligned.to_csv('rf_daily.csv')
print("Saved rf_daily.csv")

Saved rf_daily.csv
